In [5]:
import pandas as pd
from pathlib import Path

root_pth = Path("/home/gs/work/FinMem-LLM-StockTrading/Fake-Sample-Data")
news_file = root_pth/"example_input"/"Fake-News-Data-for-Each-Stock/AMZN_fake.csv"
news = pd.read_csv(news_file, sep = ",")
news.head(3)


,date,symbols,headline,body
0,2016-01-13,AMZN,Obama veers left,"On Day 13, a video message and a meeting with ..."
1,2016-01-13,AMZN,Federal Election Commission (“FEC”) Is Another...,VA Backlog Means Thousands Of Veterans Owed Mo...
2,2016-01-13,AMZN,Twenty Years of a Dictatorial Democracy : ...,Twenty Years of a Dictatorial Democracy By Ja...


In [4]:
#SEC
filing_file = root_pth/"example_input"/"filing_data.parquet"
filing = pd.read_parquet(filing_file)
filing.head(3)


,document_url,content,ticker,cik,utc_timestamp,est_timestamp,type
0,<url>,10Q report,TSLA,cik,2016-01-30 00:00:00+00:00,2016-01-29 19:00:00-05:00,10-Q
1,<url>,10Q report,TSLA,cik,2016-02-23 00:00:00+00:00,2016-02-22 19:00:00-05:00,10-Q
2,<url>,10Q report,COIN,cik,2016-02-25 00:00:00+00:00,2016-02-24 19:00:00-05:00,10-Q


In [11]:
#
import pickle
data_name = ["env_data", "filing_k", "filing_q", "news_fake", "price"]
i_data = []
for nm in data_name:
    filing_file = root_pth/"example_output"/f"{nm}.pkl"
    with open(filing_file, "rb") as fp:
        filing_q = pickle.load(fp)
        i_data.append(filing_q)
# filing_q

In [54]:
input_data = pd.DataFrame.from_dict(i_data[4], orient = "index")
input_data["filing_k"] = [{"TSLA": ""}] * input_data.shape[0] 
input_data["filing_q"] = [{"TSLA": ""}] * input_data.shape[0]
input_data["news"] = [{"TSLA": [""]}] * input_data.shape[0] 

finmem_fmt_data = "/home/gs/work/FinMem-LLM-StockTrading/data/03_model_input/tsla.pkl"
with open(finmem_fmt_data, "wb") as fp:
    pickle.dump(input_data.to_dict(orient = "index"), fp)


In [58]:
input_data.index[:20]

Index([2016-01-13, 2016-01-14, 2016-01-15, 2016-01-19, 2016-01-20, 2016-01-21,
       2016-01-22, 2016-01-25, 2016-01-26, 2016-01-27, 2016-01-28, 2016-01-29,
       2016-02-01, 2016-02-02, 2016-02-03, 2016-02-04, 2016-02-05, 2016-02-08,
       2016-02-09, 2016-02-10],
      dtype='object')

In [50]:
from langchain_community.embeddings import OllamaEmbeddings
# ollama_emb = OllamaEmbeddings(
#     model="llama3:text",
# )
ollama_emb = OllamaEmbeddings(
            model="llama3:text",
            # chunk_size=1023,
            # show_progress_bar=True,
        )
r1 = ollama_emb.embed_documents(
    [
        "Alpha is the first letter of Greek alphabet",
        "Beta is the second letter of Greek alphabet",
    ]
)
r2 = ollama_emb.embed_query(
    "What is the second letter of Greek alphabet"
)

In [57]:
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="llama3")
messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "tell me a joke"},
]
llm.invoke(messages).content

'Why did the scarecrow win an award?\n\nBecause he was outstanding in his field!\n\nHope that made you crow with laughter!'